### Import Dependencies

In [1]:
import sys
sys.path.append("../")
import pandas as pd

from utils.prompts import render
from utils.llm_client import LLMClient
from utils.logging_utils import log_llm_call
from utils.router import pick_model,should_use_reasoning_model
from IPython.display import display, Markdown

### Loading data from sample.txt file

In [2]:
file_path = '../data/raw/Incidents.txt'

incidents = []

with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# Skip header
header = [h.strip() for h in lines[0].split('|')]

for line in lines[1:]:
    if line.strip() == "":
        continue  # skip empty lines
    values = [v.strip() for v in line.split('|')]
    entry = dict(zip(header, values))
    
    # Convert numeric fields
    entry['ID'] = int(entry['ID'])
    entry['People'] = int(entry['People'])
    
    # Convert Ages to list of integers
    entry['Ages'] = [str(a.strip()) for a in entry['Ages'].split(',')]
    
    incidents.append(entry)


### Assign a Priority Score for each incidernt by COT

In [3]:
model = pick_model('openai', 'cot')
client = LLMClient('openai', model)

instruction = """
Calculate the Priority Score (1-10) for the incident based on these rules:
1. Base Score: 5.
2. Age Factor: +2 if age > 60 or < 5.
3. Life Threat: +3 if Need is "Rescue".
4. Medical: +1 if Need is "Insulin" or "Medicine".

STRICT OUTPUT RULE:
- Provide ONLY the score in the exact format shown below.
- NO reasoning, NO Incident IDs, NO labels, NO extra text.

Format:
Score: [Score]/10
"""

prompt_text, spec = render(
    'cot_reasoning.v1',
    role='emergency_triage_specialist',
    problem="{incident}"
)

print(prompt_text) 




You are emergency_triage_specialist. Apply the specified crisis-prioritization rules exactly.
Problem: {incident}

First, outline your reasoning steps briefly.
Then provide the final answer clearly marked under 'Answer:'.
Keep reasoning concise; avoid unnecessary prose.



In [4]:
incident_scores = []
for i, incident in enumerate(incidents, 1):
    full_prompt = f"""text: {prompt_text.replace("{incident}", str(incident))}
    instruction: {instruction}"""

    messages = [
        {"role": "user", "content": full_prompt}
    ]

    print(f"\n=== Incident {i} ===")
    response = client.chat(messages, temperature=0, max_tokens=spec.max_tokens)
    incident_scores.append(response['text'])
    log_llm_call('openai', model, 'cot', response['latency_ms'], response['usage'])

# updating existing data with scores
for i, incident in enumerate(incidents, 1):
    incident['Score'] = incident_scores[i-1]
    print(incident)




=== Incident 1 ===

=== Incident 2 ===

=== Incident 3 ===
{'ID': 1, 'Time': '08:00 AM', 'Area': 'Gampaha', 'People': 4, 'Ages': ['20-40'], 'Main Need': 'Water', 'Message': '"Thirsty but safe on roof. Water level stable."', 'Score': 'Score: 5/10'}
{'ID': 2, 'Time': '08:15 AM', 'Area': 'Ja-Ela', 'People': 1, 'Ages': ['75'], 'Main Need': 'Insulin', 'Message': '"Diabetic, missed dose yesterday. Feeling faint."', 'Score': 'Score: 8/10'}
{'ID': 3, 'Time': '08:20 AM', 'Area': 'Ragama', 'People': 2, 'Ages': ['10', '35'], 'Main Need': 'Rescue', 'Message': '"Water approaching neck level. Child is crying."', 'Score': 'Score: 8/10'}


### ToT - Strategy

In [5]:
model_tot = pick_model('openai', 'tot')
client_tot = LLMClient('openai', model_tot)

incidents_data_string = "\n".join([str(inc) for inc in incidents])

problem_statement = f"""
SITUATION: 
You are at Ragama with ONE rescue boat. 
There are serveral incidents to handle:
{incidents_data_string}

TRAVEL TIMES:
- Ragama to Ja-Ela: 10 mins
- Ja-Ela to Gampaha: 40 mins
(Note: Ragama to Gampaha would be 10+40 = 50 mins)

GOAL:
Maximize the total priority score saved within the shortest time. 
The boat has unlimited capacity but must stop at each location to complete the incident.

TASK:
Explore 3 distinct branches:
1. Branch 1 (Greedy): Save highest score first.
2. Branch 2 (Speed): Save closest first.
3. Branch 3 (Logistics): Save furthest first.

Compare all three and select the optimal route.
"""

prompt_text, spec = render(
    'tot_reasoning.v1',
    role='Logistics Commander',
    problem=problem_statement,
    branches='3'
)

messages = [{'role': 'user', 'content': prompt_text}]
response = client_tot.chat(messages, temperature=spec.temperature, max_tokens=spec.max_tokens)

print('ToT Response (Multiple Solution Paths):')
print('=' * 80)
print(response['text'])
log_llm_call('openai', model_tot, 'tot', response['latency_ms'], response['usage'])



ToT Response (Multiple Solution Paths):
Below is one way to “think like a Logistics Commander” by developing three alternative solution branches. In our scenario, we must rescue three incidents from Ragama with one boat. We know that:

• Incident 1 – Gampaha, 4 people, main need “Water,” Score 5/10 (time 08:00 AM)  
• Incident 2 – Ja-Ela, 1 person (elderly needs Insulin), Score 8/10 (time 08:15 AM)  
• Incident 3 – Ragama, 2 people (child and adult; rescue needed as water reaches neck level), Score 8/10 (time 08:20 AM)

Travel times provided:  
– Ragama to Ja-Ela = 10 minutes  
– Ja-Ela to Gampaha = 40 minutes  
(Thus, Ragama to Gampaha “directly” takes an effective 10+40 = 50 minutes.)

The overall goal is to “maximize the total priority score saved (i.e. 21 points in sum if all are handled) within the shortest time.”

Let’s explore each branch with its own hypothesis, steps, and an intermediate check.

──────────────────────────────
Branch 1 (Greedy: Save Highest Score First)

Hypoth